In [2]:
!pip install folium

In [3]:
import folium

In [33]:
# (위도, 경도)

# 맵의 중심점을 잡아줌
map = folium.Map(location=(35.23365594546662, 129.08112364145543),
                zoom_start=14)

mk1 = folium.Marker(location=(35.23365594546662, 129.08112364145543), popup='부산대학교')
mk1.add_to(map)

# 압축해도 ok
# folium.Marker(location=(35.23365594546662, 129.08112364145543), popup='부산대학교').add_to(map)

mk2 = folium.Marker(location=(35.2337284944008, 129.0831216804711), popup='기계기술연구동')
mk2.add_to(map)

# 한글깨짐은 저장해서 열어보면 안 깨짐(주피터 자체에서 버전업이 되면서 깨짐)
# map.save('a.html')
map

In [11]:
import urllib.request as REQ
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family']='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus']=False

In [13]:
df = pd.read_csv('../data/도로교통공단_전국_사망교통사고정보(2018).csv', encoding='euc-kr')
df

,발생년,발생년월일시,발생분,주야,요일,사망자수,사상자수,중상자수,경상자수,부상신고자수,...,도로형태_대분류,도로형태,가해자_당사자종별_대분류,가해자_당사자종별,피해자_당사자종별_대분류,피해자_당사자종별,발생위치X_UTMK,발생위치Y_UTMK,경도,위도
0,2018,2018021406,35,주,수,1,1,0,0,0,...,단일로,기타단일로,승용차,승용차,보행자,보행자,950497.0,1954702.0,126.939274,37.590388
1,2018,2018090205,23,야,일,1,1,0,0,0,...,교차로,교차로횡단보도내,승용차,승용차,보행자,보행자,953337.0,1950150.0,126.971732,37.549508
2,2018,2018121522,35,야,토,1,1,0,0,0,...,단일로,기타단일로,이륜차,이륜차,없음,없음,956967.0,1954807.0,127.012552,37.591659
3,2018,2018082311,30,주,목,1,1,0,0,0,...,단일로,기타단일로,승합차,승합차,없음,없음,958020.0,1959193.0,127.024227,37.631240
4,2018,2018071600,50,야,월,1,1,0,0,0,...,교차로,교차로횡단보도내,화물차,화물차,보행자,보행자,957606.0,1958258.0,127.019590,37.622794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3652,2018,2018052103,55,야,월,1,1,0,0,0,...,단일로,기타단일로,승용차,승용차,없음,없음,973070.0,1988564.0,127.193701,37.896529
3653,2018,2018060808,43,주,금,1,1,0,0,0,...,단일로,기타단일로,화물차,화물차,농기계,농기계,979900.0,2006212.0,127.270890,38.055765
3654,2018,2018012010,15,주,토,1,1,0,0,0,...,단일로,기타단일로,승용차,승용차,없음,없음,1001226.0,1959902.0,127.513896,37.638589
3655,2018,2018020321,5,야,토,1,1,0,0,0,...,단일로,기타단일로,승용차,승용차,승용차,승용차,1000491.0,1988419.0,127.505584,37.895621


In [14]:
df.columns

Index(['발생년', '발생년월일시', '발생분', '주야', '요일', '사망자수', '사상자수', '중상자수', '경상자수',
       '부상신고자수', '발생지시도', '발생지시군구', '사고유형_대분류', '사고유형_중분류', '사고유형', '법규위반_대분류',
       '법규위반', '도로형태_대분류', '도로형태', '가해자_당사자종별_대분류', '가해자_당사자종별',
       '피해자_당사자종별_대분류', '피해자_당사자종별', '발생위치X_UTMK', '발생위치Y_UTMK', '경도', '위도'],
      dtype='object')

# 1. 부산에서 일어난 요일별 사망자수, 사상자수 를 출력하시요

In [27]:
df2 = df[df['발생지시도']=='부산']
df2.groupby('요일')['사망자수','사상자수'].sum().loc[['월','화','수','목','금','토','일']]

<ipython-input-27-a1115f2f6545>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2.groupby('요일')['사망자수','사상자수'].sum().loc[['월','화','수','목','금','토','일']]


,사망자수,사상자수
요일,,
월,16,22
화,13,17
수,19,48
목,22,33
금,14,14
토,28,39
일,11,15


# 2. 부산 사하구 지역의 교통사고 위치를 지도로 표시하고 각마커에는 법규위반내용을 표시하시요

In [28]:
df3 = df[df['발생지시군구']=='사하구']
df3 = df3[['법규위반','위도','경도']]
df3

,법규위반,위도,경도
42,보행자 보호의무 위반,35.092209,128.960255
944,안전거리 미확보,35.106171,128.963631
1390,신호위반,35.064719,128.979537


In [29]:
map =folium.Map(location=[df3['위도'].mean(),df3['경도'].mean()], zoom_start = 13)

for a,b,c in df3.values:
    folium.Marker( location = (b,c) , popup = a).add_to(map)
    
map
# map.save('사하구 사고.html')

# 3. 부산에서 발생한 총 사망자수를 구하시요

In [17]:
df[df['발생지시도'] == '부산']['사망자수'].sum()

123

In [35]:
# len은 단순히 행이 몇개인지 보여줌

len(df[df['발생지시도'] == '부산']['사망자수'])
# df[df['발생지시도'] == '부산']['사망자수'].count()

121

# 4. 부산지역 월요일에 발생한 사상자수, 사고유형, 법규위반을 구하시요

In [19]:
df[(df['발생지시도'] == '부산')&(df['요일'] == '월')][['사상자수', '사고유형', '법규위반']]

,사상자수,사고유형,법규위반
37,1,횡단중,안전운전 의무 불이행
467,1,횡단중,안전운전 의무 불이행
468,1,기타,안전운전 의무 불이행
937,1,측면충돌,신호위반
944,1,측면충돌,안전거리 미확보
1391,1,차도통행중,과속
1392,3,측면충돌,중앙선 침범
1393,1,횡단중,과속
1816,2,공작물충돌,안전운전 의무 불이행
1821,1,횡단중,과속


# 5.횡단중 일어난 사고의 요일 발생지시도, 법규위반, 피해자_당사자종별을 구하시요

In [22]:
df[df['사고유형_중분류'] == '횡단중'][['발생지시도', '법규위반', '피해자_당사자종별']]

,발생지시도,법규위반,피해자_당사자종별
0,서울,안전운전 의무 불이행,보행자
1,서울,안전운전 의무 불이행,보행자
5,서울,안전운전 의무 불이행,보행자
6,서울,신호위반,보행자
7,서울,과속,보행자
...,...,...,...
3636,경기,보행자 보호의무 위반,보행자
3639,경기,신호위반,보행자
3643,경기,안전운전 의무 불이행,보행자
3646,경기,안전운전 의무 불이행,보행자
